# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 20 2022 10:22AM,239927,15,8157945,"Laser Pharmaceuticals, LLC",Released
1,Jun 20 2022 10:21AM,239924,15,PNC367395A,"Person & Covey, Inc.",Released
2,Jun 20 2022 10:21AM,239924,15,PNC367396A,"Person & Covey, Inc.",Released
3,Jun 20 2022 10:21AM,239924,15,PNC367397A,"Person & Covey, Inc.",Released
4,Jun 20 2022 10:21AM,239924,15,PNC367398A,"Person & Covey, Inc.",Released
5,Jun 20 2022 10:21AM,239924,15,PNC367400A,"Person & Covey, Inc.",Released
6,Jun 20 2022 10:21AM,239924,15,PNC367401A,"Person & Covey, Inc.",Released
7,Jun 20 2022 10:21AM,239924,15,PNC367403A,"Person & Covey, Inc.",Released
8,Jun 20 2022 10:21AM,239924,15,PNC367404A,"Person & Covey, Inc.",Released
9,Jun 20 2022 10:18AM,239938,10,6610184,"Citieffe, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,239933,Released,1
32,239934,Released,1
33,239936,Released,1
34,239937,Released,1
35,239938,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
239933,NaN,1.0
239934,NaN,1.0
239936,NaN,1.0
239937,NaN,1.0
239938,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
239878,0.0,1.0
239891,1.0,0.0
239892,0.0,1.0
239896,0.0,1.0
239897,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
239878,0,1
239891,1,0
239892,0,1
239896,0,1
239897,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,239878,0,1
1,239891,1,0
2,239892,0,1
3,239896,0,1
4,239897,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,239878,,1
1,239891,1,
2,239892,,1
3,239896,,1
4,239897,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 20 2022 10:22AM,239927,15,"Laser Pharmaceuticals, LLC"
1,Jun 20 2022 10:21AM,239924,15,"Person & Covey, Inc."
9,Jun 20 2022 10:18AM,239938,10,"Citieffe, Inc."
10,Jun 20 2022 10:15AM,239937,22,"NBTY Global, Inc."
11,Jun 20 2022 10:14AM,239936,22,"NBTY Global, Inc."
12,Jun 20 2022 10:12AM,239934,22,"NBTY Global, Inc."
13,Jun 20 2022 10:12AM,239933,10,"Citieffe, Inc."
14,Jun 20 2022 10:10AM,239932,22,"NBTY Global, Inc."
15,Jun 20 2022 10:09AM,239931,10,ISDIN Corporation
16,Jun 20 2022 10:09AM,239920,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 20 2022 10:22AM,239927,15,"Laser Pharmaceuticals, LLC",,1
1,Jun 20 2022 10:21AM,239924,15,"Person & Covey, Inc.",,8
2,Jun 20 2022 10:18AM,239938,10,"Citieffe, Inc.",,1
3,Jun 20 2022 10:15AM,239937,22,"NBTY Global, Inc.",,1
4,Jun 20 2022 10:14AM,239936,22,"NBTY Global, Inc.",,1
5,Jun 20 2022 10:12AM,239934,22,"NBTY Global, Inc.",,1
6,Jun 20 2022 10:12AM,239933,10,"Citieffe, Inc.",,1
7,Jun 20 2022 10:10AM,239932,22,"NBTY Global, Inc.",,1
8,Jun 20 2022 10:09AM,239931,10,ISDIN Corporation,,1
9,Jun 20 2022 10:09AM,239920,15,"Brookfield Pharmaceuticals, LLC",,69


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 10:22AM,239927,15,"Laser Pharmaceuticals, LLC",1,
1,Jun 20 2022 10:21AM,239924,15,"Person & Covey, Inc.",8,
2,Jun 20 2022 10:18AM,239938,10,"Citieffe, Inc.",1,
3,Jun 20 2022 10:15AM,239937,22,"NBTY Global, Inc.",1,
4,Jun 20 2022 10:14AM,239936,22,"NBTY Global, Inc.",1,
5,Jun 20 2022 10:12AM,239934,22,"NBTY Global, Inc.",1,
6,Jun 20 2022 10:12AM,239933,10,"Citieffe, Inc.",1,
7,Jun 20 2022 10:10AM,239932,22,"NBTY Global, Inc.",1,
8,Jun 20 2022 10:09AM,239931,10,ISDIN Corporation,1,
9,Jun 20 2022 10:09AM,239920,15,"Brookfield Pharmaceuticals, LLC",69,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 10:22AM,239927,15,"Laser Pharmaceuticals, LLC",1,
1,Jun 20 2022 10:21AM,239924,15,"Person & Covey, Inc.",8,
2,Jun 20 2022 10:18AM,239938,10,"Citieffe, Inc.",1,
3,Jun 20 2022 10:15AM,239937,22,"NBTY Global, Inc.",1,
4,Jun 20 2022 10:14AM,239936,22,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 10:22AM,239927,15,"Laser Pharmaceuticals, LLC",1.0,NaN
1,Jun 20 2022 10:21AM,239924,15,"Person & Covey, Inc.",8.0,NaN
2,Jun 20 2022 10:18AM,239938,10,"Citieffe, Inc.",1.0,NaN
3,Jun 20 2022 10:15AM,239937,22,"NBTY Global, Inc.",1.0,NaN
4,Jun 20 2022 10:14AM,239936,22,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 10:22AM,239927,15,"Laser Pharmaceuticals, LLC",1.0,0.0
1,Jun 20 2022 10:21AM,239924,15,"Person & Covey, Inc.",8.0,0.0
2,Jun 20 2022 10:18AM,239938,10,"Citieffe, Inc.",1.0,0.0
3,Jun 20 2022 10:15AM,239937,22,"NBTY Global, Inc.",1.0,0.0
4,Jun 20 2022 10:14AM,239936,22,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3598730,262.0,63.0
12,239919,1.0,0.0
15,959692,85.0,0.0
16,1439429,6.0,0.0
18,239898,1.0,0.0
19,239900,0.0,1.0
20,479824,16.0,0.0
21,479856,2.0,0.0
22,959739,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3598730,262.0,63.0
1,12,239919,1.0,0.0
2,15,959692,85.0,0.0
3,16,1439429,6.0,0.0
4,18,239898,1.0,0.0
5,19,239900,0.0,1.0
6,20,479824,16.0,0.0
7,21,479856,2.0,0.0
8,22,959739,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,262.0,63.0
1,12,1.0,0.0
2,15,85.0,0.0
3,16,6.0,0.0
4,18,1.0,0.0
5,19,0.0,1.0
6,20,16.0,0.0
7,21,2.0,0.0
8,22,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,262.0
1,12,Released,1.0
2,15,Released,85.0
3,16,Released,6.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Executing,63.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Released,262.0,1.0,85.0,6.0,1.0,0.0,16.0,2.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Executing,63.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Released,262.0,1.0,85.0,6.0,1.0,0.0,16.0,2.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Executing,63.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Released,262.0,1.0,85.0,6.0,1.0,0.0,16.0,2.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()